## Mini Project 3 Part B

## SQL Loan-Income-State Database for Loan Risk Analysis

## 100 Points (will be converted to 20 points)


In this mini-project, you will work on the Home Mortgage Disclosure Act (HMDA) dataset from US government ([link](https://www.consumerfinance.gov/data-research/hmda/historic-data/)). Specifically, you will work on the nationalwide records of 2017 as a zipped CSV file (`hmda_2017_nationwide_first-lien-owner-occupied-1-4-family-records_labels.zip`).

**<font color="red">The goal of this mini-project is to gather statistics useful to estimate the percentage of high risk applicants per state. The obtained results are presented as an SQL table accompanied by a scatter plot of average income versus average loan amount for every US state.</font>**

This goal is achieved by completing five major tasks:

- [Data Extraction (collect applicants Loan-Income per state) and Cleaning (remove missing values)](#task-1-15-pts-data-extraction-collect-applicants-loan-income-per-state-and-cleaning-remove-missing-values)
- [Statistics per State of the Loan-Income-Applicants Data (min, max, average)](#task-2-30-pts-statistics-per-state-of-the-loan-income-applicants-data-min-max-average)
- [Creation of a Basic SQL Table from the Loan-Income-State Data](#task-3-15-pts-creation-of-a-basic-sql-table-from-the-loan-income-state-data)
- [Detection of High-Loan and High-Risk Applicants and Extension of the Basic SQL Table with this Information](#task-4-20-pts-detection-of-high-loan-and-high-risk-applicants-and-extension-of-the-basic-sql-table-with-this-information)
- [Creation of a Scatter Plot of High-Risk States (average income versus average loan amount)](#task-5-5-pts-creation-of-a-scatter-plot-of-high-risk-states-average-income-versus-average-loan-amount)


### Note: The first step to complete is to import the necessary libraries indicated below


Please make sure you run the next cell to import the necessary libraries before you start working on the assigned tasks.


In [83]:
# For reading csv in zip files
import zipfile
from io import TextIOWrapper
import csv

# For plots
from matplotlib import pyplot as plt

# For connecting to MySQL
from mysql.connector import connect, Error
import time

# user-defined module to get all the states' abbreviations
from util import state_abbrs

### Task 1 (15 pts): Data Extraction (collect applicants Loan-Income per state) and Cleaning (remove missing values)


#### Subtask 1.1 (5 pts): Extract the data from the zipped CSV file


Write a function `read_zip_csv` following the code template provided below. This function should take three inputs and produce one output as indicated next. You just need to complete the missing code in the cell containing the string `"""__Your_Code_Here__"""`.

- **Input 1**: a string `zip_file_name` representing the path to a zip file.
- **Input 2**: a string `csv_file_name` representing the name of a CSV file within the zip file.
- **Input 3**: a list of string `column_names` specifying the names of the columns of interest.
- **Output**: a list of lists of strings `data`, where each inner list represents a row containing only the specified columns according to **Input 3** from the CSV file.

Note: In order to complete the missing code, it is imperative that you understand the intended purpose of the code template.


In [84]:
def read_zip_csv(zip_file_name, csv_file_name, column_names):
    """
    Read a csv file from a zip file, and return data in the specified columns
    IN: zip_file_name, str, the name of the zip file
        csv_file_name, str, the name of the csv file in the zip file
        column_names, list[str], the names of the columns to extract
    OUT: data, list[list[str]], the data from the csv file in the specified columns
    """

    data = []
    # use zipfile to open the zip file directly
    with zipfile.ZipFile(zip_file_name) as z:
        with z.open(csv_file_name) as f:
            # use csv to read the zipped csv file
            reader = csv.DictReader(TextIOWrapper(f, 'utf-8'))
            # iterate through the rows of the csv file
            for idx, row in enumerate(reader, start=1):
                # extract the data in the specified columns
                data.append([row[col] for col in column_names])

                if idx % 1_000_000 == 0:
                    print(f"Read {idx} rows")
    return data

In this mini-project, we are interested in the following three columns of the HMDA dataset: `state_abbr`, `loan_amount_000s` and `applicant_income_000s`. Please run the cell below to extract the information in these three columns before moving on to the next subtask.


In [85]:
if __name__ == "__main__":
    # The code in this cell calls the function defined above.
    # It loads the three columns into a Python list named `data`.
    # Before proceeding, it is a good idea to run this code to see a snapshot of the data.
    data = read_zip_csv(
        "hmda_2017_nationwide_first-lien-owner-occupied-1-4-family-records_labels.zip",
        "hmda_2017_nationwide_first-lien-owner-occupied-1-4-family-records_labels.csv",
        ['state_abbr', 'loan_amount_000s', 'applicant_income_000s']
    )
    # print the first 5 rows of the data
    print(data[:5])

FileNotFoundError: [Errno 2] No such file or directory: 'hmda_2017_nationwide_first-lien-owner-occupied-1-4-family-records_labels.zip'

#### Subtask 1.2 (10 pts): Clean the extracted data


Write a function `clean_data` that takes two inputs and produces two outputs as indicated below. You just need to complete the missing code in the cell containing the string `"""__Your_Code_Here__"""`.

- **Input 1**: a list of lists of strings `data` as extracted from the CSV file. Each sublist represents a row with the relevant columns.
- **Input 2**: a list of functions `cleaners`. Each function converts values in a specific column to a desired type, with the order of functions in the list matching the column order in `data`. Each function should have the following signature:
  - **input**: a string `column_value` to be cleaned.
  - **output**: A cleaned value (of type string, int, or float, depending on the column).
- **Output 1**: a list of tuples `cleaned_data`, where each tuple represents a row of cleaned data in the desired types.
- **Output 2**: a pair of integers `(n_rows_cleaned, n_rows_discarded)` representing the number of rows cleaned and the number discarded, respectively.

Note: If a value in a row fails to be cleaned, the entire row should be discarded.


In [86]:
def clean_data(data, cleaners):
    """
    Clean the data using the specified cleaners
    IN: data, list[list[str]], the data to clean
        cleaners, list[function(str) -> any], the functions to clean each column
    OUT: cleaned_data, list[tuple], the cleaned data
         (n_rows_cleaned, n_rows_discarded), tuple(int, int) the number of rows cleaned and discarded
    """

    cleaned_data = []
    n_rows_cleaned = 0
    n_rows_discarded = 0
    for row in data:
        # use try-except to filter out the rows that cannot be cleaned
        try:
            # clean each value in the row
            cleaned_row = tuple(cleaner(val)
                                for cleaner, val in zip(cleaners, row))

        except ValueError as e:
            # some values in the row are invalid, so discard the row
            n_rows_discarded += 1

        else:
            # all values in the row are valid, so append the cleaned row
            n_rows_cleaned += 1
            cleaned_data.append(cleaned_row)

    return cleaned_data, (n_rows_cleaned, n_rows_discarded)

In this mini-project, we need to clean the data in the following ways:

- `state_abbr`: convert to uppercase letters, and if it is not a valid 2-letter state abbreviation, discard the row.
- `loan_amount_000s`: convert to integers, and if it is empty, discard the row.
- `applicant_income_000s`: convert to integers, and if it is empty, discard the row.


In [87]:
def clean_state_abbr(column_value):
    """
    Clean a state abbreviation
    IN: column_value, str, the state abbreviation to clean
    OUT: str, the cleaned state abbreviation in uppercase
    EXCEPTION: ValueError, if the state abbreviation is not valid
    """
    # state_abbrs is a set of all the state abbreviations
    # you can use it to check if a state abbreviation is valid
    if column_value.upper() not in state_abbrs:
        raise ValueError(f"Invalid state abbreviation: {column_value}")
    return column_value.upper()


def clean_loan_amount(column_value):
    """
    Clean a loan amount
    IN: column_value, str, the loan amount to clean
    OUT: int, the cleaned loan amount
    EXCEPTION: ValueError, if the loan amount is not an integer
    """
    # convert to integer
    try:
        return int(column_value)
    except ValueError:
        raise ValueError(f"Invalid loan amount: {column_value}")


def clean_applicant_income(column_value):
    """
    Clean an applicant income
    IN: column_value, str, the applicant income to clean
    OUT: int, the cleaned applicant income
    EXCEPTION: ValueError, if the applicant income is not an integer
    """
    # convert to integer
    try:
        return int(column_value)
    except ValueError:
        raise ValueError(f"Invalid applicant income: {column_value}")

Please run the cell below to clean the three columns before moving on to the next task.


In [88]:
if __name__ == "__main__":
    # The code in this cell calls the functions defined above.
    # It cleans the data using the specified cleaners.
    # Before proceeding, it is a good idea to run this code to see a snapshot of the cleaned data.
    cleaned_data, (n_rows_cleaned, n_rows_discarded) = clean_data(
        data,
        [
            clean_state_abbr,
            clean_loan_amount,
            clean_applicant_income,
        ],
    )
    # print the first 5 rows of the cleaned data
    print(cleaned_data[:5])
    # print the number of rows cleaned and discarded
    print(n_rows_cleaned, n_rows_discarded)

NameError: name 'data' is not defined

### Task 2 (30 pts): Statistics per State of the Loan-Income-Applicants Data (min, max, average)


#### Subtask 2.1 (12 pts): Check the statistics of the cleaned data and store the results in dictionaries


Write a function `check_data` that takes three inputs and produces one output as indicated below. You just need to complete the missing code in the cell containing the string `"""__Your_Code_Here__"""`.

- **Input 1**: a list of tuples `cleaned_data` representing cleaned rows from the CSV file.
- **Input 2**: a list of functions `checkers`. Each function checks a specific column, with the order of functions in the list matching the column order in`cleaned_data`. Each function should have the following signature:
  - **input a**: a cleaned data value (string, int, or float, depending on the column).
  - **input b**: a dictionary representing the statistical summary of the column up to, but excluding, the current row, based on the column type.
  - **output**: A dictionary representing the updated statistical summary of the column, including the current row.
- **Input 3**: a list of dictionaries `initial_stats`, where each dictionary represents the initial statistical summary for a column. The order of dictionaries in the list should match the order of columns in `cleaned_data`.
- **Output**: a list of tuples `final_stats`, where each tuple represents the final statistical summary of a column. The order of tuples in the list should match the order of columns in `cleaned_data`.


In [89]:
def check_data(cleaned_data, checkers, initial_stats):
    """
    Check the data using the specified checkers
    IN: cleaned_data, list[tuple], the cleaned data
        checkers, list[function(any, dict) -> dict], the functions to check each column
        initial_stats, list[dict], the initial statistics for each column
    OUT: final_stats, list[dict], the final statistics for each column
    """

    # initialize the final statistics
    final_stats = initial_stats.copy()
    # iterate through the cleaned data
    for row in cleaned_data:
        # update the statistics using the checkers
        for checker, val, stats in zip(checkers, row, final_stats):
            stats.update(checker(val, stats))
    return final_stats

In this mini-project we are going to check the data as follows:

- `state_abbr`: Count the occurrences of each state in the data.
- `loan_amount_000s`: Record the **dictionary** keys `min`, `max`, and `total` loan amounts (`total` can be used to calculate the average loan amount).
- `applicant_income_000s`: Record the **dictionary** keys `min`, `max`, and `total` applicant incomes (`total` can be used to calculate the average applicant income).

The initial statistical summary for each column is a **dictionary** with the following structure:

```python
state_abbr_stat = {abbr: 0 for abbr in state_abbrs}
loan_amount_000s_stat = {'min': float('inf'), 'max': float('-inf'), 'total': 0}
applicant_income_000s_stat = {'min': float('inf'), 'max': float('-inf'), 'total': 0}
```


In [90]:
def check_state_abbr(column_value, stats):
    """
    Check a state abbreviation, updating the count of each state abbreviation
    IN: column_value, str, the state abbreviation to check
        stats, dict{state_abbr: int}, the current count of each state abbreviation
    OUT: stats, dict{state_abbr: int}, the updated count of each state abbreviation
    """
    # check if the state abbreviation is in the dictionary
    if column_value not in stats:
        # if not, add it
        stats[column_value] = 1
    else:
        # if it is, increment the count
        stats[column_value] += 1

    return stats


def check_loan_amount(column_value, stats):
    """
    Check a loan amount, updating the minimum, maximum, and total loan amount
    IN: column_value, int, the loan amount to check
        stats, dict{min: int, max: int, total: int}, the current statistics for the loan amount
    OUT: dict{min: int, max: int, total: int}, the updated statistics for the loan amount
    """
    # update the min value if necessary
    if column_value < stats['min']:
        stats['min'] = column_value

    # update the max value if necessary
    if column_value > stats['max']:
        stats['max'] = column_value

    # update the total value
    stats['total'] += column_value

    return stats


def check_applicant_income(column_value, stats):
    """
    Check an applicant income, updating the minimum, maximum, and total applicant income
    IN: column_value, int, the applicant income to check
        stats, dict, the current statistics for the applicant income
    OUT: dict, the updated statistics for the applicant income
    """
    # update min value
    if column_value < stats['min']:
        stats['min'] = column_value

    # update max value
    if column_value > stats['max']:
        stats['max'] = column_value

    # update total value
    stats['total'] += column_value

    return stats

Before moving on to the next subtask, please run the cell below to check the statistics of the cleaned data.


In [91]:
if __name__ == "__main__":
    # The code in this cell calls the functions defined above.
    # It checks the cleaned data using the specified checkers.
    # Before proceeding, it is a good idea to run this code to see the final statistics
    final_stats = check_data(
        cleaned_data,
        [
            check_state_abbr,
            check_loan_amount,
            check_applicant_income,
        ],
        [
            {state: 0 for state in state_abbrs},
            {'min': float('inf'), 'max': float('-inf'), 'total': 0},
            {'min': float('inf'), 'max': float('-inf'), 'total': 0},
        ],
    )
    # print the statistics for state abbreviation
    print(final_stats[0])
    # print the statistics for loan amount
    print(final_stats[1])
    # print the statistics for applicant income
    print(final_stats[2])

NameError: name 'cleaned_data' is not defined

#### Subtask 2.2 (4 pts): Plot the bar chart of the number of applicants by state


Write a function `plot_applicants_by_state` that takes one input and produces a plot as indicated below. You just need to complete the missing code in the cell containing the string `"""__Your_Code_Here__"""`.

- **Input**: a dictionary `state_abbr_stat` representing the number of applicants per state.
  - **key**: a state abbreviation.
  - **value**: the number of applicants.
- **Plot**: a bar chart displaying the number of applicants per state, sorted by the applicant count in descending order. Ties should be broken by sorting state abbreviations in ascending order.
  - **x-axis**: state abbreviation.
  - **y-axis**: the number of applicants.


In [ ]:
def plot_applicants_by_state(state_abbr_stat):
    """
    Plot the number of applicants by state
    IN: state_abbr_stat, dict{str: int}, the number of applicants by state
    """

    # sort the states by the number of applicants from greatest to least
    state_abbr_stat = dict(
        sorted(state_abbr_stat.items(), key=lambda x: x[1], reverse=True))

    # extract the states and counts
    states = state_abbr_stat.keys()
    counts = state_abbr_stat.values()

    # increase the figure size to avoid overlapping labels
    plt.figure(figsize=(15, 5))

    # draw a bar plot
    # if you forgot how to specify a bar plot, you can check Mini-Project 2
    plt.bar(states, counts)

    # set the labels and title
    plt.xlabel('State Abbreviation')
    plt.ylabel('Number of applicants')
    plt.title('Number of applicants by State')
    plt.show()

Before moving on to the next subtask, please run the cell below to plot the bar chart of the number of applicants by state.


In [ ]:
if __name__ == "__main__":
    # The code in this cell calls the function defined above.
    # It plots the number of applicants by state.
    # Before proceeding, it is a good idea to run this code to see the plot.
    plot_applicants_by_state(final_stats[0])

#### Subtask 2.3 (12 pts): Plot Average Loan Amount and Average Applicant Income by State


##### Subsubtask 2.3.1 (4 pts): Calculate the total loan amount and total applicant income by state


Write a function `get_total_loan_income_by_state` that takes one input and produces one output as indicated below. You just need to complete the missing code in the cell containing the string `"""__Your_Code_Here__"""`.

- **Input**: a list of tuples `cleaned_data` extracted and cleaned from the CSV file.
- **Output**: a dictionary `total_loan_income_by_state` representing the total loan amount and total applicant income by state.
  - **key**: a state abbreviation.
  - **value**: a pair of integers `(total_loan_amount, total_applicant_income)`.


In [ ]:
from collections import defaultdict


def get_total_loan_income_by_state(cleaned_data):
    """
    Get the total loan amount and total applicant income by state
    IN: cleaned_data, list[tuple], the cleaned data
    OUT: total_loan_income_by_state, dict{str: tuple(int, int)}, the total loan amount and total applicant income by state
    """

    # initialize the total loan amount and total applicant income by state
    total_loan_income_by_state = defaultdict(lambda: [0, 0])

    # iterate through the cleaned data to update the total loan amount and total applicant income by state
    for state_abbr, loan_amount, applicant_income in cleaned_data:
        # update the total loan amount and total applicant income for the state
        total_loan_income_by_state[state_abbr][0] += loan_amount
        total_loan_income_by_state[state_abbr][1] += applicant_income

    return total_loan_income_by_state

Before moving on to the next subsubtask, please run the cell below to calculate the total loan amount and total applicant income by state.


In [ ]:
if __name__ == "__main__":
    # The code in this cell calls the function defined above.
    # It gets the total loan amount and total applicant income by state.
    # Before proceeding, it is a good idea to run this code to see the total loan amount and total applicant income by state.
    total_loan_income_by_state = get_total_loan_income_by_state(cleaned_data)
    # print the total loan amount and total applicant income by state
    for state_abbr, (total_loan, total_income) in total_loan_income_by_state.items():
        print(f"{state_abbr}: {total_loan}, {total_income}")

##### Subsubtask 2.3.2 (2 pts): Calculate the average loan amount and average applicant income


Write a function `get_avg_loan_income_by_state` that takes three inputs and produces two outputs as indicated below. You just need to complete the missing code in the cell containing the string `"""__Your_Code_Here__"""`.

- **Input 1**: a dictionary `total_loan_income_by_state` representing the total loan amount and total applicant income by state.
- **Input 2**: a dictionary `state_abbr_stat` representing the number of applicants by state.
- **Input 3**: a pair of integers `(total_loan_amount, total_applicant_income)` representing the total loan amount and total applicant income across all applicants.
- **Output 1**: a dictionary `avg_loan_income_by_state` representing the average loan amount and average applicant income by state.
  - **key**: a state abbreviation.
  - **value**: a tuple `(avg_loan_amount, avg_applicant_income)`.
- **Output 2**: a tuple `(avg_loan_amount, avg_applicant_income)` representing the average loan amount and average applicant income across all applicants.


In [ ]:
def get_avg_loan_income_by_state(total_loan_income_by_state, state_abbr_stat, total_loan_income):
    """
    Get the average loan amount and average applicant income by state
    IN: total_loan_income_by_state, dict{str: tuple(int, int)}, the total loan amount and total applicant income by state
        state_abbr_stat, dict{str: int}, the number of applicants by state
        total_loan_income, tuple(int, int), the total loan amount and total applicant income of all applicants
    OUT: avg_loan_income_by_state, dict{str: tuple(float, float)}, the average loan amount and average applicant income by state
         avg_loan_income, tuple(float, float), the average loan amount and average applicant income of all applicants.
    """

    # initialize the average loan amount and average applicant income by state
    avg_loan_income_by_state = {}

    # calculate the average loan amount and average applicant income by state
    for state_abbr, (total_loan, total_income) in total_loan_income_by_state.items():
        # calculate the average loan amount and average applicant income for the state
        avg_loan = total_loan / state_abbr_stat[state_abbr]
        avg_income = total_income / state_abbr_stat[state_abbr]
        avg_loan_income_by_state[state_abbr] = (avg_loan, avg_income)

    # calculate the average loan amount and average applicant income of all applicants
    avg_loan_income = tuple(
        total / sum(state_abbr_stat.values()) for total in total_loan_income)

    return avg_loan_income_by_state, avg_loan_income

Before moving on to the next subsubtask, please run the cell below to calculate the average loan amount and average applicant income by state.


In [ ]:
if __name__ == "__main__":
    # The code in this cell calls the function defined above.
    # It gets the average loan amount and average applicant income by state.
    # Before proceeding, it is a good idea to run this code to see the average loan amount and average applicant income by state.
    avg_loan_income_by_state, avg_loan_income = get_avg_loan_income_by_state(
        total_loan_income_by_state,
        final_stats[0],
        (final_stats[1]['total'], final_stats[2]['total']),
    )
    # print the average loan amount and average applicant income by state
    print(avg_loan_income_by_state)
    # print the average loan amount and average applicant income of all applicants
    print(avg_loan_income)

##### Subsubtask 2.3.3 (2 pts): Find the ratio of the average loan amount and average applicant income by state


Write a function `get_avg_loan_income_ratio_range` that:

- **Input**: a dictionary `avg_loan_income_by_state` representing the average loan amount and average applicant income by state.
- **Output**: a dictionary `state_avg_loan_income_ratio` representing the ratio of the average loan amount to the average applicant income by state.
  - **key**: a state abbreviation.
  - **value**: the ratio of the average loan amount to the average applicant income.

Note:

- Given a specific applicant, the ratio of the loan amount to the applicant income is a measure of the applicant's financial health. A higher ratio means it is more risky (more likely to a financial failure) for the applicant to take a loan.
- When calculating the ratio of the average loan amount to the average applicant income by state, one can multiply the number of applicants to both the numerator and the denominator to get ratio of the total loan amount to total applicant income by state. This measures the financial health of the state as a whole.


In [ ]:
def get_avg_loan_income_ratio_range(avg_loan_income_by_state):
    """
    Get the ratio of the average loan amount and average applicant income by state
    IN: avg_loan_income_by_state, dict{str: tuple(float, float)}, the average loan amount and average applicant income by state
    OUT: state_avg_loan_income_ratio, dict{str: float}, the ratio of the average loan amount and average applicant income by state
    """

    # initialize the ratio of the average loan amount and average applicant income by state
    state_avg_loan_income_ratio = {}

    # iterate through the average loan amount and average applicant income by state to calculate the ratio
    for state_abbr, (avg_loan, avg_income) in avg_loan_income_by_state.items():
        # calculate the ratio of the average loan amount and average applicant income for the state
        state_avg_loan_income_ratio[state_abbr] = avg_loan / avg_income

    return state_avg_loan_income_ratio

Before moving on to the next subtask, please run the cell below to find the ratio of the average loan amount and average applicant income by state.


In [ ]:
if __name__ == "__main__":
    # The code in this cell calls the function defined above.
    # It gets the ratio of the average loan amount and average applicant income by state.
    # Before proceeding, it is a good idea to run this code to see the ratio of the average loan amount and average applicant income by state.
    state_avg_loan_income_ratio = get_avg_loan_income_ratio_range(
        avg_loan_income_by_state)
    # print the ratio of the average loan amount and average applicant income by
    print(state_avg_loan_income_ratio)

##### Subsubtask 2.3.4 (4 pts): 2D scatter Plot for the average loan amount and average applicant income by state


Write a function `plot_avg_loan_income_by_state` that takes four inputs and produces a plot as indicated below. You just need to complete the missing code in the cell containing the string `"""__Your_Code_Here__"""`.

- **Input 1**: a dictionary `avg_loan_income_by_state` representing the average loan amount and average applicant income by state.
- **Input 2**: a dictionary `state_abbr_stat` representing the number of applicants by state.
- **Input 3**: a tuple `(avg_loan_amount, avg_applicant_income)` representing the average loan amount and average applicant income for all applicants.
- **Input 4**: a dictionary `state_avg_loan_income_ratio` representing the ratio of the average loan amount to the average applicant income by state.
- **Plot**: a scatter plot of the average loan amount and average applicant income by state.
  - **x-axis**: average loan amount.
  - **y-axis**: average applicant income.
  - **size**: the number of applicants, where the radius of each circle is proportional to the square root of the number of applicants. You may use some scaling factor to make the circles well sized.
  - **color**: the ratio of the average loan amount to the average applicant income, with the colormap set to `copper_r`.
  - **vertical line**: the average loan amount for all applicants (in a dashed black line).
  - **horizontal line**: the average applicant income for all applicants (in a dashed black line).


`plt.scatter()` can be used to create a scatter plot. It takes the following arguments:

- `x`: a list of numbers representing the x-coordinates of the data points.
- `y`: a list of numbers representing the y-coordinates of the data points.
- `s`: a list of numbers representing the size (area) of the data points.
- `c`: a list of numbers representing the color of the data points.
- `cmap`: a string representing the colormap, which determines how the color values `c` are mapped to colors.

A typical usage of `plt.scatter()` is as follows:

```python

# specify the x, y, s, c
x_list = [1, 2.5, 0.8, -1.2]
y_list = [3, -1.5, 2.8, 0.2]
s_list = [100, 400, 800, 200]
c_list = [0.1, 0.5, 0.8, 0.2]

# create the scatter plot, and specify the colormap
plt.scatter(x_list, y_list, s=s_list, c=c_list, cmap='copper_r')

# add the colorbar to the plot
plt.colorbar()

# show the plot
plt.show()
```


To add vertical reference lines and horizontal reference lines, you can use `plt.axvline()` and `plt.axhline()`, respectively. For example:

- To add a vertical line at x=1.5, with a black dashed line:
  ```python
  plt.axvline(x=1.5, color='k', linestyle='--')
  ```
  `x` is the x-coordinate of the vertical line, `color` `k` is short for black, and `linestyle` `--` specifies a dashed line.
- To add a horizontal line at y=-0.3, with a black dashed line:
  ```python
  plt.axhline(y=-0.3, color='k', linestyle='--')
  ```
  `y` is the y-coordinate of the horizontal line, `color` `k` is short for black, and `linestyle` `--` specifies a dashed line.


In [ ]:
def plot_avg_loan_income_by_state(avg_loan_income_by_state, state_abbr_stat, avg_loan_income, state_avg_loan_income_ratio):
    """
    Plot the average loan amount and average applicant income by state
    IN: avg_loan_income_by_state, dict{str: tuple(float, float)}, the average loan amount and average applicant income by state
        state_abbr_stat, dict{str: int}, the number of applicants by state
        avg_loan_income, tuple(float, float), the average loan amount and average applicant income of all applicants
        state_avg_loan_income_ratio, dict{str: float}, the ratio of the average loan amount and average applicant income by state
    """

    # extract the states, average loan amounts, average applicant incomes, sizes, and colors
    states = []
    avg_loans = []
    avg_incomes = []
    sizes = []
    colors = []

    for state_abbr, (avg_loan, avg_income) in avg_loan_income_by_state.items():
        states.append(state_abbr)
        avg_loans.append(avg_loan)
        avg_incomes.append(avg_income)
        sizes.append(state_abbr_stat[state_abbr])
        colors.append(state_avg_loan_income_ratio[state_abbr])

    # increase the size of the plot to avoid crowding
    plt.figure(figsize=(15, 10))
    # use ggplot style. it's gray background, easy for colors to stand out
    plt.style.use('ggplot')

    # plot the average loan amount and average applicant income by state
    plt.scatter(avg_loans, avg_incomes, s=sizes,
                c=colors, cmap='coolwarm', alpha=0.6)

    # add horizontal and vertical lines for the average loan amount and average applicant income of all applicants
    plt.axhline(avg_loan_income[0], c='r', ls='--', lw=1)
    plt.axvline(avg_loan_income[1], c='b', ls='--', lw=1)

    # add state names to the plot
    for i, state_abbr in enumerate(states):
        plt.text(avg_loans[i], avg_incomes[i], state_abbr,
                 fontsize=12, c='white', ha='center', va='center')

    # add labels, title, and colorbar to the plot
    plt.xlabel('Average Loan Amount')
    plt.ylabel('Average Applicant Income')
    plt.title('Average Loan Amount and Average Applicant Income by State')
    plt.colorbar()
    plt.show()

Before moving on to the next subtask, please run the cell below to plot the 2D scatter plot for the average loan amount and average applicant income by state.


In [82]:
if __name__ == "__main__":
    # The code in this cell calls the function defined above.
    # It plots the average loan amount and average applicant income by state.
    # Before proceeding, it is a good idea to run this code to see the plot.
    plot_avg_loan_income_by_state(
        avg_loan_income_by_state, final_stats[0], avg_loan_income, state_avg_loan_income_ratio)

NameError: name 'avg_loan_income_by_state' is not defined

### Task 3 (15 pts): Creation of a Basic SQL Table from the Loan-Income-State Data


#### The first step to complete the following tasks is to define two helper functions described below.


Before moving on to the next subtask, please run the cell below to define the helper functions.


In [ ]:
WAIT_TIME = 10


def connect_to_db(db_name):
    """
    Connect to the database
    IN: db_name, str, the name of the database in MySQL
    OUT: mydb, the database connection
    EXCEPTION: TimeoutError, if the database is not running
    """

    num_of_tries = 0
    while num_of_tries < WAIT_TIME:
        try:
            mydb = connect(host="127.0.0.1", user="root", password = "Khoimai2012", db=db_name)
            if mydb.is_connected():
                print("Connected")
                break
        except Error as e:
            print(e)

        num_of_tries += 1
        print("Waiting for MySQL to start")
        time.sleep(1)

    if num_of_tries >= WAIT_TIME:
        raise TimeoutError("MySQL is not running")

    return mydb

In [ ]:
def do_query(cursor, query):
    """
    Execute a query and print the results
    IN: cursor, the cursor object
        query, str, the query to be executed
    OUT: list[tuple], the results of the query
    """
    cursor.execute(query)
    res = cursor.fetchall()
    print(cursor.column_names)
    for row in res:
        print(row)
    return res

In [ ]:
if __name__ == "__main__":
    database_name = "proj3"  # change this to your database name
    mydb = connect_to_db(database_name)
    cursor = mydb.cursor()

#### Subtask 3.1 (6 pts): Get the specification of an SQL Table schema encoded by a string


Write a function `get_table_specifications_sql` that takes one input and produces one output as indicated below. You just need to complete the missing code in the cell containing the string `"""__Your_Code_Here__"""`.

- **INPUT**: a string `table_name` representing the name of the table.
- **OUTPUT**: a string `table_specifications_sql` encoding the `CREATE TABLE` statements for the SQL database.

Specifically, the schema of the SQL database is as follows:

- `id`: an integer primary key, auto-incremented.
- `state_abbr`: a string of fixed length 2, and it cannot be null.
- `loan_amount`: an integer, and it cannot be null.
- `applicant_income`: an integer, and it cannot be null.


In [ ]:
def get_table_specifications_sql(table_name):
    """
    Get the create table statements for the SQL database
    IN: table_name, str, the name of the table
    OUT: table_specifications_sql, str, the create table statements for the SQL database
    """

    table_specifications_sql = f"""
    CREATE TABLE {table_name} (
        id INT AUTO_INCREMENT PRIMARY KEY,
        state_abbr VARCHAR(2) not NULL,
        loan_amount INT not NULL,
        applicant_income INT not NULL
    )
    """
    return table_specifications_sql

In [ ]:
if __name__ == "__main__":
    table_name = "applicants"  # change this to your table name
    table_specifications_sql = get_table_specifications_sql(table_name)
    print(table_specifications_sql)
    do_query(cursor, table_specifications_sql)

#### Subtask 3.2 (9 pts): Insert Loan-Income-State data into the SQL Table


Write a function `insert_data` that takes four inputs and produces one output as indicated below. You just need to complete the missing code in the cell containing the string `"""__Your_Code_Here__"""`.

- **Input 1**: a connection `mydb` to the SQL database.
- **Input 2**: a cursor `cursor` to the SQL database.
- **Input 3**: a string `table_name` representing the name of the table.
- **Input 4**: a list of tuples `cleaned_data` containing the cleaned data from the CSV file.
- **Output**: an integer representing the number of rows inserted into the table.

Note: make sure to commit the changes to the database.


In [ ]:
def insert_data(mydb, cursor, table_name, cleaned_data):
    """
    Insert the cleaned data into the table
    IN: mydb, the database connection
        cursor, the cursor object to the database
        table_name, str, the name of the table
        cleaned_data, list[tuple], the cleaned data
    OUT: n_rows_inserted, int, the number of rows inserted into the table
    """

    insert_sql = f"""
    INSERT INTO {table_name} (state_abbr, loan_amount, applicant_income)
    VALUES (%s, %s, %s)
    """

    # Insert the data in batches of 10,000 rows to avoid memory errors
    n_rows_inserted = 0
    for i in range(0, len(cleaned_data), 10_000):
        cursor.executemany(insert_sql, cleaned_data[i:i+10_000])
        mydb.commit()
        n_rows_inserted += cursor.rowcount
        print(f"Inserted {n_rows_inserted} rows")

    return n_rows_inserted

In [ ]:
if __name__ == "__main__":
    mydb.rollback()
    n_rows_inserted = insert_data(mydb, cursor, table_name, cleaned_data)
    print(n_rows_inserted)

### Task 4 (35 pts): Detection of High-Loan and High-Risk Applicants and Extension of the Basic SQL Table with this Information


#### Subtask 4.1 (15 pts): Flag high loan applicants


##### Subsubtask 4.1.1 (4 pts): Add a new column `high_loan_flag` to the SQL Table


Write a function `get_add_high_loan_flag_sql` that takes one input and produces one output as indicated below. You just need to complete the missing code in the cell containing the string `"""__Your_Code_Here__"""`.

- **Input**: a string `table_name` representing the name of the table.
- **Output**: a string `add_high_loan_flag_sql` encoding the `ALTER TABLE` statement to add a new column `high_loan_flag` to the SQL database.

Specifically, the `high_loan_flag` is a boolean column that defaults to False and cannot be null.

Hint: You can specify a column as boolean type in MySQL using the `BOOLEAN` keyword.


In [ ]:
def get_add_high_loan_flag_sql(table_name):
    """
    Get the alter table statement to add a new column high_loan_flag to the SQL database
    IN: table_name, str, the name of the table
    OUT: add_high_loan_flag_sql, str, the alter table statement to add a new column high_loan_flag to the SQL database
    """

    return f"""
    ALTER TABLE {table_name}
    ADD COLUMN high_loan_flag BOOLEAN DEFAULT FALSE NOT NULL
    """

In [ ]:
if __name__ == "__main__":
    add_high_loan_flag_sql = get_add_high_loan_flag_sql(table_name)
    print(add_high_loan_flag_sql)
    do_query(cursor, add_high_loan_flag_sql)
    mydb.commit()

##### Subsubtask 4.1.2 (7 pts): Update the new column `high_loan` in the SQL Table


Write a function `get update_high_loan_flag_sql` that takes two inputs and produces one output as indicated below. You just need to complete the missing code in the cell containing the string `"""__Your_Code_Here__"""`.

- **Input 1**: a string `table_name` representing the name of the table.
- **Input 2**: an integer `loan_threshold` representing the threshold for the loan amount to be considered a high loan. The default value is 700.
- **Output**: a string `update_high_loan_flag_sql` encoding the `UPDATE` statement to set the `high_loan_flag` to True for rows with a loan amount greater than the threshold.

Note: A loan amount is considered a high loan if it is strictly greater than the threshold.

Hint: You can use the `UPDATE` statement to set the value of the `high_loan_flag` column to True for rows with a loan amount greater than the threshold.

The syntax for the `UPDATE` statement is as follows:

```sql
UPDATE table_name
SET column_name = updated_value
WHERE condition;
```


In [ ]:
def get_update_high_loan_flag_sql(table_name, loan_threshold=700):
    """
    Get the update statement to set the high_loan_flag to True if the loan amount is greater than the threshold
    IN: table_name, str, the name of the table
        loan_threshold, int, the threshold of the loan amount to be considered as high loan
    OUT: update_high_loan_flag_sql, str, the update statement to set the high_loan_flag to True if the loan amount is greater than the threshold
    """

    return f"""
    UPDATE {table_name}
    SET high_loan_flag = TRUE
    WHERE loan_amount > {loan_threshold}
    """

In [ ]:
if __name__ == "__main__":
    update_high_loan_flag_sql = get_update_high_loan_flag_sql(table_name)
    print(update_high_loan_flag_sql)
    do_query(cursor, update_high_loan_flag_sql)
    mydb.commit()

##### Subsubtask 4.1.3 (4 pts): Find the average applicant income of high loan applicants


Write a function `get_avg_income_high_loan_sql` that takes one input and produces one output as indicated below. You just need to complete the missing code in the cell containing the string `"""__Your_Code_Here__"""`.

- **Input**: a string `table_name` representing the name of the table.
- **Output**: a string `avg_income_high_loan_sql` encoding the SQL query to find the average applicant income of high loan applicants.

Hint: Boolean values can be compared using `boolean_column = 1` for True, and `boolean_column = 0` for False.


In [ ]:
def get_avg_income_high_loan_sql(table_name):
    """
    Get the query to find the average applicant income of high loan applicants
    IN: table_name, str, the name of the table
    OUT: avg_income_high_loan_sql, str, the query to find the average applicant income of high loan applicants
    """

    return f"""
    SELECT AVG(applicant_income)
    FROM {table_name}
    WHERE high_loan_flag = TRUE
    """

In [ ]:
if __name__ == "__main__":
    avg_income_high_loan_sql = get_avg_income_high_loan_sql(table_name)
    print(avg_income_high_loan_sql)
    do_query(cursor, avg_income_high_loan_sql)

#### Subtask 4.2 (20 pts): Label applicants with a Low/Moderate/High Risk flag


##### Subsubtask 4.2.1 (4 pts): Add a new column `risk_flag` to the SQL Table


Write a function `get_add_risk_flag_sql` that takes one input and produces one output as indicated below. You just need to complete the missing code in the cell containing the string `"""__Your_Code_Here__"""`.

- **Input**: a string `table_name` representing the name of the table.
- **Output**: a string `add_risk_flag_sql` encoding the `ALTER TABLE` statement to add a new column `risk_flag` to the SQL database.

Specifically, the `risk_flag` is a string column of variable length up to two characters, defaults to "H" (High risk), and cannot be null.


In [ ]:
"""
Write a function `get_add_risk_flag_sql` that:

- **Input**: a string `table_name` representing the name of the table.
- **Output**: a string `add_risk_flag_sql` encoding the alter table statement to add a new column `risk_flag` to the SQL database.

Specifically, the `risk_flag` is a string column of variable length up to two characters, and it cannot be null.
"""


def get_add_risk_flag_sql(table_name):
    """
    Get the alter table statement to add a new column risk_flag to the SQL database
    IN: table_name, str, the name of the table
    OUT: add_risk_flag_sql, str, the alter table statement to add a new column risk_flag to the SQL database
    """

    return f"""
    ALTER TABLE {table_name}
    ADD COLUMN risk_flag VARCHAR(2) NOT NULL
    """

In [ ]:
if __name__ == "__main__":
    add_risk_flag_sql = get_add_risk_flag_sql(table_name)
    print(add_risk_flag_sql)
    do_query(cursor, add_risk_flag_sql)
    mydb.commit()

##### Subsubtask 4.2.2 (10 pts): Update the new column `risk_flag` in the SQL Table


Write a function `get_update_risk_flag_sql` that takes two inputs and produces one output as indicated below. You just need to complete the missing code in the cell containing the string `"""__Your_Code_Here__"""`.

- **Input 1**: a string `table_name` representing the name of the table.
- **Input 2**: a pair of integers `(low_mod_threshold, mod_high_threshold)` representing the thresholds for the ratio of the average loan amount to average applicant income, to be considered as "L", "M", or "H". Defaults to (0.4, 0.8).
- **Output**: a string `update_risk_flag_sql` encoding the SQL query to update the risk flag of the applicants.

Hint: You can use the `CASE` statement in SQL to update the column based on the condition.

```sql
...
CASE
    WHEN condition1 THEN value1
    WHEN condition2 THEN value2
    ...
    ELSE default_value
END
```


In [ ]:
def get_update_risk_flag_sql(table_name, low_mod_high_thresholds=(0.4, 0.8)):
    """
    Get the update statement to set the risk_flag based on the ratio of the average loan amount and average applicant income
    IN: table_name, str, the name of the table
        low_mod_high_thresholds, tuple(float, float), the thresholds of the ratio of the average loan amount and average applicant income to be considered as "L"/"M"/"H"
    OUT: update_risk_flag_sql, str, the update statement to set the risk_flag based on the ratio of the average loan amount and average applicant income
    """

    low, mod = low_mod_high_thresholds
    return f"""
    UPDATE {table_name}
    SET risk_flag = CASE
        WHEN loan_amount / applicant_income < {low} THEN 'L'
        WHEN loan_amount / applicant_income < {mod} THEN 'M'
        ELSE 'H'
    END
    """

In [ ]:
if __name__ == "__main__":
    update_risk_flag_sql = get_update_risk_flag_sql(table_name)
    print(update_risk_flag_sql)
    do_query(cursor, update_risk_flag_sql)
    mydb.commit()

##### Subsubtask 4.2.3 (6 pts): Find the percentage of high-risk applicants per state


Write a function `get_high_risk_percentage_sql` that takes one input and produces one output as indicated below. You just need to complete the missing code in the cell containing the string `"""__Your_Code_Here__"""`.

- **Input**: a string `table_name` representing the name of the table.
- **Output**: a string `high_risk_percentage_sql` encoding the SQL query to find the percentage of high-risk applicants per state.

Note: the percentage of high-risk applicants is the number of high-risk applicants divided by the total number of applicants per state.

The SQL query should return a table with the following columns:

- `state_abbr`: the state abbreviation.
- `high_risk_percentage`: as two decimal floating-point numbers. For example, 0.25 represents 25%.

Hint: You may need to nest the `SELECT` statement in the `FROM` clause to calculate the total number of applicants per state.

```sql
SELECT temp_table.state_abbr, ...
FROM (
    SELECT ...
    FROM table_name
) AS temp_table
```


In [ ]:
"""
Write a function `get_high_risk_percentage_sql` that:

- **Input**: a string `table_name` representing the name of the table.
- **Output**: a string `high_risk_percentage_sql` encoding the SQL query to find the percentage of high-risk applicants per state.

Note: the percentage of high-risk applicants is the number of high-risk applicants divided by the total number of applicants per state.

The SQL query should return a table with the following columns:

- `state_abbr`: the state abbreviation.
- `high_risk_percentage`: as integer representing the percentage of high-risk applicants.
"""


def get_high_risk_percentage_sql(table_name):
    """
    Get the query to find the percentage of high-risk applicants per state
    IN: table_name, str, the name of the table
    OUT: high_risk_percentage_sql, str, the query to find the percentage of high-risk applicants per state
    """

    return f"""
    SELECT state_abbr, 
           SUM(CASE WHEN risk_flag = 'H' THEN 1 ELSE 0 END) / COUNT(*) * 100 AS high_risk_percentage
    FROM {table_name} 
    GROUP BY state_abbr
    """

In [ ]:
if __name__ == "__main__":
    high_risk_percentage_sql = get_high_risk_percentage_sql(table_name)
    print(high_risk_percentage_sql)
    high_risk_percentage = do_query(cursor, high_risk_percentage_sql)

### Task 5 (5 pts): Creation of a Scatter Plot of High-Risk States (average income versus average loan amount)


Write a function `plot_high_risk_percentage` that takes four inputs and produces a plot as indicated below. You just need to complete the missing code in the cell containing the string `"""__Your_Code_Here__"""`.

- **Input 1**: a dictionary `avg_loan_income_by_state` representing the average loan amount and average applicant income by state.
- **Input 2**: a dictionary `state_abbr_stat` representing the number of applicants by state.
- **Input 3**: a tuple `(avg_loan_amount, avg_applicant_income)` representing the average loan amount and average applicant income for all applicants.
- **Input 4**: a list of tuples `high_risk_percentage` representing the percentage of high-risk applicants per state.
- **Plot**: a scatter plot of the average loan amount and average applicant income by state.
  - **x-axis**: average loan amount.
  - **y-axis**: average applicant income.
  - **size**: the number of applicants, where the radius of each circle is proportional to the square root of the number of applicants. You may use some scaling factor to make the circles well sized.
  - **color**: the percentage of high-risk applicants per state, with the colormap set to `copper_r`.
  - **vertical line**: the average loan amount for all applicants (in a dashed black line).
  - **horizontal line**: the average applicant income for all applicants (in a dashed black line).


In [ ]:
def plot_high_risk_percentage(avg_loan_income_by_state, state_abbr_stat, avg_loan_income, high_risk_percentage):
    """
    Plot the percentage of high-risk applicants per state
    IN: avg_loan_income_by_state, dict{str: tuple(float, float)}, the average loan amount and average applicant income by state
        state_abbr_stat, dict{str: int}, the number of applicants by state
        avg_loan_income, tuple(float, float), the average loan amount and average applicant income for all applicants
        high_risk_percentage, list[tuple(str, float)], the percentage of high-risk applicants per state
    """

    # convert the high_risk_percentage to a dictionary
    high_risk_percentage_dict = dict(high_risk_percentage)


    # extract the states, average loan amounts, average applicant incomes, sizes, and colors
    states = []
    avg_loans = []
    avg_incomes = []
    sizes = []
    colors = []

    for state_abbr, (avg_loan, avg_income) in avg_loan_income_by_state.items():
        states.append(state_abbr)
        avg_loans.append(avg_loan)
        avg_incomes.append(avg_income)
        sizes.append(state_abbr_stat[state_abbr])
        colors.append(high_risk_percentage_dict.get(state_abbr, 0))
    

    # increase the size of the plot to avoid crowding
    plt.figure(figsize=(15, 10))
    # use ggplot style. it's gray background, easy for colors to stand out
    plt.style.use('ggplot')

    # plot the average loan amount and average applicant income by state
    plt.scatter(avg_loans, avg_incomes, s=sizes,
                c=colors, cmap='coolwarm', alpha=0.6)
    # add horizontal and vertical lines for the average loan amount and average applicant income of all applicants
    plt.axhline(avg_loan_income[0], c='r', ls='--', lw=1)

    # add state names to the plot
    for i, state_abbr in enumerate(states):
        plt.text(avg_loans[i], avg_incomes[i], state_abbr,
                 fontsize=12, c='white', ha='center', va='center')

    # add labels, title, and colorbar to the plot
    plt.xlabel('Average Loan Amount')
    plt.ylabel('Average Applicant Income')
    plt.title('Average Loan Amount and Average Applicant Income by State')
    plt.colorbar()
    plt.show()

### Note: Do not forget to close the connection to the database.


In [ ]:
if __name__ == "__main__":
    cursor.close()
    mydb.close()